# Answering Business Questions using SQL

We will use the chinook database for this project. The chinook database is provided as a SQLite database file called chinook.db.


![](chinook-schema.svg)

In [34]:
from sqlalchemy.engine import create_engine
import sqlite3
from pandas.io import sql
import subprocess
%reload_ext sql

In [35]:
%sql sqlite:///chinook.db

'Connected: @chinook.db'

## Overview of the Data

Let us start by getting familiar with the data

In [36]:
%%sql
SELECT 
     name,
     type
FROM sqlite_master
WHERE type IN ('table','view');

 * sqlite:///chinook.db
Done.


name,type
albums,table
sqlite_sequence,table
artists,table
customers,table
employees,table
genres,table
invoices,table
invoice_items,table
media_types,table
playlists,table


## Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and the task is selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that do not have any tracks in the store right now - we have the artist names, and the genre of the music they produce.
|Artist | Genre |
|:----------:|:----------:|
|Regal | Hip-hop|
|Red Tone | Punk|
|Meteor and The Girls | Pop|
|Slim Jim Bites |Blues| 

The record label specializes in artists from the USA, and they have given Chinook some money to advertise new albums in the USA, so we are interested in finding out which genres sell the best in the USA.

In [37]:
%%sql
WITH genre_sold_usa AS
(
    SELECT 
       g.Name genre,
       SUM(il.Quantity) num_tracks
      FROM genres g
      INNER JOIN tracks t on t.GenreId = g.GenreId
      INNER JOIN invoice_items il on il.TrackId = t.TrackId
      INNER JOIN invoices i on i.InvoiceId = il.InvoiceId
      INNER JOIN customers c on c.CustomerId = i.CustomerId
     WHERE c.country = 'USA'
     GROUP BY 1
     ORDER BY 2
)

SELECT 
    gsu.genre Genre,
    gsu.num_tracks abs_tracks
FROM genre_sold_usa gsu;

 * sqlite:///chinook.db
Done.


Genre,abs_tracks
Science Fiction,1
Easy Listening,3
Rock And Roll,3
Heavy Metal,4
Hip Hop/Rap,4
Soundtrack,4
Alternative,5
Pop,5
Sci Fi & Fantasy,5
Drama,6


From the above, it can be said that Rock sells best in the USA, with a total of 157 tracks sold so far. However, Rock is not available in the record label.
Below is a list of the number of tracks sold according to the genres available in the record label, in a decreasing order, on the basis of no. of tracks sold.
1. Alternative & Punk - 50
2. Blues - 15
3. Pop - 5
4. Hip Hop/Rap - 4

Based on the list, we should purchase the albums of artists Red Tone, Slim Jim Bites, and Meteor and The Girls respectively, since their genres sell well.

## Analyzing Employee Sales Performace

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [38]:
%%sql

SELECT 
   e.FirstName || ' ' || e.LastName name,
   e.Title Title,
   e.BirthDate Birth_Date,
   e.HireDate Hire_Date,
   max(i.InvoiceDate) Latest_invoice_date,
   ROUND(SUM(i.Total), 2) amount_sales
  FROM employees e
  LEFT JOIN customers c on e.EmployeeId = c.SupportRepId
  LEFT JOIN invoices i on i.CustomerId = c.CustomerId

  WHERE Title = 'Sales Support Agent'
  GROUP BY 1
  ORDER BY 2

 * sqlite:///chinook.db
Done.


name,Title,Birth_Date,Hire_Date,Latest_invoice_date,amount_sales
Jane Peacock,Sales Support Agent,1973-08-29 00:00:00,2002-04-01 00:00:00,2013-12-22 00:00:00,833.04
Margaret Park,Sales Support Agent,1947-09-19 00:00:00,2003-05-03 00:00:00,2013-12-09 00:00:00,775.4
Steve Johnson,Sales Support Agent,1965-03-03 00:00:00,2003-10-17 00:00:00,2013-12-05 00:00:00,720.16


## Analyzing Sales by Country

The next task is to analyze the sales data for customers from each country. The country value from customers table will be used for this analysis, and ignore the country from the billing address in the invoice table.

We should calculate data for each country on the
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Since there are a number of countries with only one customer, these customers should be grouped as "Other" in the analysis.

In [39]:
%%sql

WITH no_of_cstmrs AS
(
    SELECT 
       country Country,
       count(FirstName) No_of_customers
     FROM customers
      GROUP BY 1
      ORDER BY 2 DESC
)

SELECT * FROM no_of_cstmrs

 * sqlite:///chinook.db
Done.


Country,No_of_customers
USA,13
Canada,8
France,5
Brazil,5
Germany,4
United Kingdom,3
Portugal,2
India,2
Czech Republic,2
Sweden,1


In [40]:
%%sql

WITH no_of_cstmrs AS
(
    SELECT 
       country Country,
       count(FirstName) No_of_customers
     FROM customers
      GROUP BY 1
      ORDER BY 2 DESC
),

total_sales_value AS
(SELECT c.country Country,
       ROUND(SUM(i.total), 2) Total_Value_Sales,
       ROUND(AVG(i.total), 2) Avg_Order_Value,
       ROUND(SUM(i.total)/(SELECT COUNT(DISTINCT(CustomerId)) FROM invoices), 2) Avg_Sales_Value_per_Customer
 
 FROM customers c
INNER JOIN invoices i ON i.CustomerId = c.CustomerId
GROUP BY 1
ORDER BY 2 DESC),

sales_data_1 AS
(SELECT noc.Country,
       noc.No_of_customers,
       tsv.Total_Value_Sales,
       tsv.Avg_Sales_Value_per_Customer,
       tsv.Avg_Order_Value
  FROM no_of_cstmrs noc  
INNER JOIN total_sales_value tsv ON tsv.Country=noc.Country
 WHERE noc.No_of_customers > 1),

sales_data_2 AS
(SELECT CASE
       WHEN noc.No_of_customers=1 THEN "Others"
       END AS
       Country,
       noc.No_of_customers,
       ROUND(AVG(tsv.Total_Value_Sales),2) Total_Value_Sales,
       ROUND(AVG(tsv.Avg_Sales_Value_per_Customer),2) AVG_Sales_Value_per_Customer,
       ROUND(AVG(tsv.Avg_Order_Value),2) AVG_Order_Value
  FROM no_of_cstmrs noc  
INNER JOIN total_sales_value tsv ON tsv.Country=noc.Country
 WHERE noc.No_of_customers = 1),

total_sales_data AS
(SELECT * 
FROM sales_data_1
UNION
SELECT * 
FROM sales_data_2)

SELECT *
  FROM total_sales_data
ORDER BY 3 DESC;

 * sqlite:///chinook.db
Done.


Country,No_of_customers,Total_Value_Sales,Avg_Sales_Value_per_Customer,Avg_Order_Value
USA,13,523.06,8.87,5.75
Canada,8,303.96,5.15,5.43
France,5,195.1,3.31,5.57
Brazil,5,190.1,3.22,5.43
Germany,4,156.48,2.65,5.59
United Kingdom,3,112.86,1.91,5.37
Czech Republic,2,90.24,1.53,6.45
Portugal,2,77.24,1.31,5.52
India,2,75.26,1.28,5.79
Others,1,40.29,0.68,5.75


## Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:
- purchase a whole album
- purchase a collection of one or more individual tracks

The store does not let customers purchase a whole album, and then add individuals tracks to the same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchase strategy to save money. The strategy considered is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is quite common to have "edge cases" which prevent us from getting a 100% accurate answer to the question. In this instance, we have two edge cases to consider:
- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to the purchase.

In the first case, since the analysis is concerned with maximizing revenue, we can safely ignore albums consisting of a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this also.

In order to answer the question, we are going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be same for all tracks.

In [41]:
%%sql

SELECT max(InvoiceId) total_invoice
    FROM invoice_items;

 * sqlite:///chinook.db
Done.


total_invoice
412


From the above table we observe that there are a total of 412 invoices. Further, we can compare the tracks from invoice with that of albums

In [42]:
%%sql

WITH com_prchs AS
(
    SELECT 
       il.InvoiceId invoice_id, 
       il.TrackId Prchs_tracks, 
       t.AlbumId album_id
      FROM tracks t
    INNER JOIN invoice_items il on il.TrackId = t.TrackId
    ORDER BY 1
)

SELECT *
 FROM com_prchs;

 * sqlite:///chinook.db
Done.


invoice_id,Prchs_tracks,album_id
1,2,2
1,4,3
2,6,1
2,8,1
2,10,1
2,12,1
3,16,4
3,20,4
3,24,5
3,28,5


From the above table it can be observed that if the tracks belonging to each distinct invoice belongs to the same album it is an Album Purchase. If these tracks belong to different albums it is Non Albu Purchase.

Below, we can categorize these invoices into Album and Non Album Purchase. This can be done by counting the number of distinct albums belonging to each invoice. Then we compare the Distinct Albums and Number of Albums belonging to each invoice.

In [44]:
%%sql

WITH com_prchs AS
(
    SELECT 
       il.InvoiceId invoice_id, 
       il.TrackId Prchs_tracks, 
       t.AlbumId album_id
      FROM tracks t
    INNER JOIN invoice_items il on il.TrackId = t.TrackId
    ORDER BY 1
)

SELECT 
   invoice_id, 
   COUNT(DISTINCT(album_id)) Distinct_album_id,
   COUNT(album_id) No_of_album_id,
   CASE
   WHEN COUNT(album_id) - COUNT(DISTINCT(album_id)) <= 2 THEN "Non Album Purchase"
   ELSE "Album Purchase"
   END AS Category
  FROM com_prchs
 GROUP BY 1;

 * sqlite:///chinook.db
Done.


invoice_id,Distinct_album_id,No_of_album_id,Category
1,2,2,Non Album Purchase
2,1,4,Album Purchase
3,2,6,Album Purchase
4,5,9,Album Purchase
5,11,14,Album Purchase
6,1,1,Non Album Purchase
7,1,2,Non Album Purchase
8,1,2,Non Album Purchase
9,1,4,Album Purchase
10,1,6,Album Purchase


### Category: Not an Album Purchase

Here we calculate the Percentage of Non Album Purchases for the company.

In [51]:
%%sql

WITH com_prchs AS
(SELECT il.InvoiceId invoice_id, il.TrackId Pur_tracks, t.AlbumId album_id
  FROM tracks t
INNER JOIN invoice_items il ON il.TrackId = t.TrackId
ORDER BY 1
),

ctgr_prchs AS
(SELECT invoice_id, COUNT(DISTINCT(album_id)) Distinct_album_id, 
       COUNT(album_id) No_of_album_id,
       CASE
       WHEN COUNT(album_id) - COUNT(DISTINCT(album_id)) <= 2 THEN "Non Album Purchase"
       ELSE "Album Purchase"
       END AS Category 
  FROM com_prchs
GROUP BY 1
)

SELECT (SELECT MAX(InvoiceId)
          FROM invoice_items) TOTAL_invoice_id,
       COUNT(invoice_id) No_of_invoice_id,
       (SELECT ROUND(CAST(COUNT(invoice_id) AS FLOAT) * 100 / (SELECT MAX(invoice_id)
                                            FROM invoice_items), 2)) Perc_invoice
  FROM ctgr_prchs
WHERE Category = "Non Album Purchase";

 * sqlite:///chinook.db
Done.


TOTAL_invoice_id,No_of_invoice_id,Perc_invoice
412,213,51.7


From the above table, it can be seen that nearly half of the purchases, i.e., 213 out of 412 are Non Album Purchase.

### Category: Album Purchase

Here we calculate the Percentage of Album Purchase for the company.

In [53]:
%%sql

WITH com_prchs AS
(SELECT il.InvoiceId invoice_id, il.TrackId Pur_tracks, t.AlbumId album_id
  FROM tracks t
INNER JOIN invoice_items il ON il.TrackId = t.TrackId
ORDER BY 1
),

ctgr_prchs AS
(SELECT invoice_id, COUNT(DISTINCT(album_id)) Distinct_album_id, 
       COUNT(album_id) No_of_album_id,
       CASE
       WHEN COUNT(album_id) - COUNT(DISTINCT(album_id)) <= 2 THEN "Non Album Purchase"
       ELSE "Album Purchase"
       END AS Category 
  FROM com_prchs
GROUP BY 1
)

SELECT (SELECT MAX(InvoiceId)
          FROM invoice_items) TOTAL_invoice_id,
       COUNT(invoice_id) No_of_invoice_id,
       (SELECT ROUND(CAST(COUNT(invoice_id) AS FLOAT) * 100 / (SELECT MAX(invoice_id)
                                            FROM invoice_items), 2)) Perc_invoice
  FROM ctgr_prchs
WHERE Category = "Album Purchase";

 * sqlite:///chinook.db
Done.


TOTAL_invoice_id,No_of_invoice_id,Perc_invoice
412,199,48.42


Here, it can be seen that nearly 48%, i.e., 199 out of 412 purchase are Album Purchases.

From the above two intepretations, it can be seen that Non Album Purchases slightly sell better. The store should consider buying popular tracks from albums rather than buying the whole album.